In [ ]:
from scripts.models import TransformerModel
model = TransformerModel(
    n_dims=4,
    n_positions=201,
    n_embd=256,
    n_layer=12,
    n_head=8,
    pred_type="regression",
).cuda()

In [56]:
from scripts.main_utils import gen_dataloader
from scripts.tasks import get_task_sampler

task_sampler = get_task_sampler(
    task_name="linear_regression",
    batch_size=2,
    n_points=7,
    n_dims=4,
    n_dims_truncated=1,
    device="cuda",
)

real_task = task_sampler()
xs, ys = real_task.xs.float(), real_task.ys.float()
n_loops = 1; horizon_start = max(0, n_loops - 20)
xs, ys, xs.shape, ys.shape

(tensor([[[-0.9282,  0.0000,  0.0000,  0.0000],
          [ 0.4507,  0.0000,  0.0000,  0.0000],
          [-3.0299,  0.0000,  0.0000,  0.0000],
          [-0.2710,  0.0000,  0.0000,  0.0000],
          [-0.3976,  0.0000,  0.0000,  0.0000],
          [ 1.5787,  0.0000,  0.0000,  0.0000],
          [ 0.2183,  0.0000,  0.0000,  0.0000]],
 
         [[ 1.0496,  0.0000,  0.0000,  0.0000],
          [-0.2318,  0.0000,  0.0000,  0.0000],
          [ 2.3027,  0.0000,  0.0000,  0.0000],
          [ 0.2870,  0.0000,  0.0000,  0.0000],
          [-0.6655,  0.0000,  0.0000,  0.0000],
          [ 1.3583,  0.0000,  0.0000,  0.0000],
          [ 0.2276,  0.0000,  0.0000,  0.0000]]], device='cuda:0'),
 tensor([[ 1.5218, -0.7389,  4.9676,  0.4443,  0.6519, -2.5883, -0.3578],
         [ 0.5869, -0.1296,  1.2876,  0.1605, -0.3721,  0.7595,  0.1273]],
        device='cuda:0'),
 torch.Size([2, 7, 4]),
 torch.Size([2, 7]))

In [57]:
y_pred = model(xs, ys, add_inputs_embeds=False) 
print(y_pred)
loss = (ys - y_pred).square().mean()
loss

tensor([[ 0.3433, -0.2220,  0.6188, -0.0318,  0.1309, -0.3208, -0.1465],
        [-0.3215,  0.0329, -0.3724, -0.2268,  0.2844, -0.2894, -0.1307]],
       device='cuda:0', grad_fn=<SelectBackward0>)


tensor(2.2576, device='cuda:0', grad_fn=<MeanBackward0>)

tensor(0.2006, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
model = TransformerModel(
    n_dims=4,
    n_positions=101,
    n_embd=256,
    n_layer=12,
    n_head=8,
    pred_type="regression",
).cuda()